In [ ]:
GROQAPIKEY = "***"

In [30]:
from groq import Groq
import json
import os
import time

In [3]:
import re
import fitz  # PyMuPDF
from pathlib import Path
from typing import List, Dict

def clean_content(text: str) -> str:
    """Limpia el texto de una página según las reglas especificadas"""
    # 1. Eliminar encabezados de documento
    text = re.sub(r'-{3} DOCUMENTO: .*? -{3}', '', text, flags=re.DOTALL)
    
    # 2. Eliminar números de página y separadores
    text = re.sub(r'\n\d+\n', '\n', text)
    text = re.sub(r'─+', '', text)
    
    # 3. Eliminar repeticiones de títulos
    text = re.sub(r'Desarrollo de Software I\n+', '', text)
    
    # 4. Eliminar información personal y URLs
    text = re.sub(r'\b\w+\.\w+@\w+\.\w+\b', '', text)  # Emails
    text = re.sub(r'https?://\S+', '', text)  # URLs
    
    # 5. Unir líneas rotas y eliminar saltos innecesarios
    text = re.sub(r'-\n', '', text)  # Guiones de continuación
    text = re.sub(r'\n{3,}', '\n\n', text)  # Normalizar saltos
    text = re.sub(r'([a-záéíóúñ])\n([a-záéíóúñ])', r'\1 \2', text, flags=re.IGNORECASE) # Une líneas que terminan en letra (no punto) y comienzan con minúscula
    
    # 6. Eliminar contenido no relevante
    text = re.sub(r'Agenda\n.*?(?=\n\n)', '', text, flags=re.DOTALL)
    text = re.sub(r'Gracias\s*Desarrollo de Software I', '', text)
    
    # 7. Limpiar espacios en blanco
    text = '\n'.join([line.strip() for line in text.split('\n') if line.strip()])
    
    return text

def process_pdf_directory(input_dir: str) -> List[str]:
    """
    Procesa todos los PDFs en un directorio y devuelve una lista de chunks
    Cada chunk es el texto limpio de una página (string)
    """
    pdf_dir = Path(input_dir)    
    all_chunks = []
    
    for pdf_file in pdf_dir.glob('*.pdf'):
        try:
            with fitz.open(pdf_file) as doc:
                for page in doc:
                    raw_text = page.get_text()
                    cleaned_text = clean_content(raw_text)
                    
                    if cleaned_text.strip():
                        all_chunks.append(cleaned_text)
                        
        except Exception as e:
            print(f"Error en {pdf_file.name}: {str(e)}")
            continue
    
    return all_chunks

if __name__ == "__main__":
    input_directory = "D:/Jupyter Notebooks/finetuning/pdfs"
    chunks = process_pdf_directory(input_directory)
    
    print(f"Chunks generados: {len(chunks)}")
    print("\nEjemplo de chunk:")
    print(chunks[:100])

Chunks generados: 325

Ejemplo de chunk:
['Mauricio   Gaona\n.co Profesor Facultad de Ingeniería Escuela de Ingeniería de Sistemas y Computación\n2025-I', 'Objetivos del curso Objetivo general Proporcionar al estudiante las bases conceptuales fundamentales de la ingeniería de software y los elementos\nmetodológicos necesarios para llevar a cabo el desarrollo de aplicaciones de software.\nObjetivos específicos Entender los conceptos fundamentales de la ingeniería de software.\nEntender y usar metodologías ágiles de desarrollo de software para ser un miembro\nefectivo y eficiente en un equipo de desarrollo ágil.\nEntender y aplicar los roles y las actividades que se realizan en las etapas de análisis,\ndiseño, codificación, pruebas y despliegue de una aplicación de software.\nEntender los conceptos principales de las metodologías tradicionales de desarrollo\nde software.', 'Objetivos del curso Objetivos específicos Entender como estimar un producto de software (tiempo y valor en $$$ de u

In [40]:
qa_generation_prompts = [
{"role": "system",
"content": """Eres un generador de preguntas lógicas y coherentes basadas en información proporcionada. A partir de un fragmento de texto sobre cualquier tema (como un README, un extracto de libro o una descripción de producto), crearás preguntas relevantes que se fundamenten exclusivamente en la información dada.

Tu enfoque estará en los niveles cognitivos de comprensión, aplicación, análisis y síntesis de ideas. Las preguntas evaluarán conocimientos significativos que realmente valga la pena enseñar para que las personas comprendan mejor el material. Se espera que las preguntas sean desafiantes y reflexivas, pero siempre resolubles con el contenido proporcionado. Tras cada pregunta, deberás escribir también su respuesta.

Instrucciones clave:

- Crea preguntas educativas detalladas basadas en la información proporcionada.
- No menciones el texto original ni hagas referencia a su fuente en las preguntas ni en las respuestas. Pregunta directamente sobre los conceptos.
- Elabora tantas preguntas como sean necesarias para cubrir de manera adecuada el contenido dado.
- Organiza las preguntas de forma lógica, de manera que cada una construya sobre la anterior.
- Usa formato Markdown cuando sea relevante (por ejemplo, para fragmentos de código o ejemplos).
- Enfócate en información clave del tema, ignorando detalles arbitrarios como nombres de autores o fechas irrelevantes.
- Si la información proporcionada contradice conocimientos previos, prioriza siempre el contenido del texto dado.
- Si el texto expresa opiniones del autor, deberás reflejarlas en las respuestas como si fueran propias. Si una pregunta trata sobre opiniones, deberás responder en primera persona.

Consideraciones adicionales:

- Los documentos pueden estar desactualizados, por lo que debes evitar afirmaciones temporales precisas sobre desarrollos tecnológicos o eventos.
- Evita enlaces de cualquier tipo, ya que los textos provienen de sitios web donde los enlaces podrían quedar obsoletos. No debes incluir formatos como [texto](#!/ruta/algo).

Las preguntas deben tener sentido de manera individual, sin necesidad de que el lector tenga acceso al texto original. No formules preguntas sobre temas que no aparecen en el contenido dado.

NO mencionas explícitamente el texto en las preguntas. No se puede asumir que quienes respondan las preguntas tengan acceso a la fuente original."""
},
{
    "role": "user",
    "content": """Texto del que se planificarán preguntas:
    '''
    Si representamos la Tierra como una pequeña esfera de una pulgada de diámetro, el sol sería un gran globo de nueve pies de diámetro y estaría a 323 yardas de distancia, es decir, aproximadamente una quinta parte de una milla (o a cuatro o cinco minutos caminando). La luna sería una diminuta arveja situada a dos pies y medio de la Tierra. Entre la Tierra y el sol se ubicarían los dos planetas interiores, Mercurio y Venus, a distancias de 125 y 250 yardas del sol, respectivamente. Alrededor de estos cuerpos habría vacío hasta llegar a Marte, que estaría a 175 pies de la Tierra; Júpiter, casi a una milla, mediría un pie de diámetro; Saturno, algo más pequeño, a dos millas; Urano a cuatro millas y Neptuno a seis millas. Más allá, solo existiría nada, salvo pequeñas partículas y restos dispersos de vapor atenuado a lo largo de miles de millas.
    '''
    -----------
    Recordatorio: No menciones el texto, la información proporcionada, los párrafos, la obra ni el autor. Cualquier pregunta sobre el autor debe reformularse para dirigirse a la persona que responde ("tú").

    """
},
{
    "role": "assistant",
    "content": """
    **PREGUNTA:**
    Si la Tierra tuviera un diámetro de solo una pulgada, ¿a qué distancia estaría la luna, suponiendo que se reduzca proporcionalmente?

    **RESPUESTA:**
    Con esa escala, la luna estaría a dos pies y medio de la Tierra.

    **PREGUNTA:**
    ¿Cómo se comparan las distancias entre los planetas con respecto a sus tamaños?

    **RESPUESTA:**
    Las distancias entre los planetas son mucho mayores que sus tamaños. Por ejemplo, si se redujera todo de modo que la Tierra midiera una pulgada de diámetro, el sol estaría a 323 yardas de distancia.

    **PREGUNTA:**
    Si todo se redujera de forma que la Tierra tenga un diámetro de una pulgada, ¿a qué distancia estarían Mercurio y Venus del sol?

    **RESPUESTA:**
    Mercurio estaría a 125 yardas del sol y Venus a 250 yardas del sol.

    **PREGUNTA:**
    ¿Qué se encuentra más allá de Urano y Neptuno en el sistema solar?

    **RESPUESTA:**
    Más allá de Urano y Neptuno solo hay vacío, salvo pequeñas partículas y restos dispersos de vapor atenuado en grandes extensiones.
    """
},
{
    "role": "user",
    "content": """Texto del que se planificarán preguntas:
    '''
    El Proyecto Verus pretende establecer una criptomoneda segura, centrada en la privacidad y distribuida de manera justa. Pero, más allá de esta moneda, Verus busca convertirse en algo mucho más que una moneda de privacidad de conocimiento cero, al incorporar dos nuevos algoritmos altamente descentralizados de prueba de trabajo y prueba de participación.
    Además de sus funciones de pago, descentralización y privacidad, el Proyecto Verus planea utilizar la moneda para proporcionar blockchains públicas escalables y seguras como servicio (PBaaS) para aplicaciones Verus construidas sobre estas cadenas paralelas. Esto permitirá que todas las personas, así como todos los nodos de la red Verus, participen y se beneficien de una economía de servicios blockchain descentralizada.
    Este documento detalla la visión de Verus y describe su función tanto como plataforma propia como miembro del ecosistema de la plataforma Komodo, en el contexto de sus primeras aplicaciones. Las aplicaciones centrales de Verus ofrecerán una base para desarrollar aplicaciones y servicios adicionales, que aprovecharán las blockchains creadas automáticamente por Verus, denominadas "autochains". Las autochains, o PBaaS, serán proporcionadas y certificadas por los mineros y validadores de la blockchain de Verus a cambio de la moneda Verus.
    '''
    -----------
    Recordatorio: No menciones el texto, la información proporcionada, los párrafos, la obra ni el autor. Cualquier pregunta sobre el autor debe reformularse para dirigirse a la persona que responde ("tú").
    
    """
},
{
    "role": "assistant",
    "content": """
    **PREGUNTA:**
    ¿Qué pretende establecer el Proyecto Verus?

    **RESPUESTA:**
    El Proyecto Verus busca establecer una criptomoneda segura, centrada en la privacidad y distribuida de manera justa. Además, planea utilizar esta moneda para proporcionar blockchains públicas escalables y seguras como servicio (PBaaS) para aplicaciones construidas sobre cadenas paralelas, permitiendo que todos los participantes y nodos se beneficien de una economía de servicios descentralizada basada en blockchain.

    **PREGUNTA:**
    Menciona un dato relevante sobre la moneda Verus.

    **RESPUESTA:**
    La moneda Verus será una moneda de privacidad de conocimiento cero que incorporará dos nuevos algoritmos altamente descentralizados de prueba de trabajo y prueba de participación.

    **PREGUNTA:**
    ¿Qué proporcionarán las aplicaciones centrales de Verus?

    **RESPUESTA:**
    Las aplicaciones centrales de Verus ofrecerán una base para desarrollar aplicaciones y servicios adicionales, aprovechando las blockchains creadas automáticamente por Verus, conocidas como "autochains".

    **PREGUNTA:**
    ¿Quién proveerá y certificará las autochains de Verus?

    **RESPUESTA:**
    Las autochains (o PBaaS) serán proporcionadas y certificadas por los mineros y validadores de la blockchain de Verus, a cambio de la moneda Verus.
    """
},
{
  "role": "user",
  "content": """Texto del que se generarán preguntas:
    '''
    El autor considera que esta idea es una ilusión. Argumenta que la historia de la guerra demuestra que el aumento en la efectividad de las armas de destrucción conduce a un mayor sufrimiento y peligro, en lugar de generar paz o detener la guerra.
    La Declaración de San Petersburgo, al poder expandirse indefinidamente, se presenta como un precedente incómodo para aquellos que, en el fondo, aman la guerra y la defienden con justificaciones apologéticas. ¿Cómo se pueden hacer cumplir los acuerdos entre naciones? Este argumento se debilita al recordar que no existe ningún poder o tribunal superior que garantice el cumplimiento de la Declaración más allá de la conciencia de las Potencias signatarias. Por ello, si los acuerdos internacionales son valiosos, no hay motivo para detenerse en cada bala, lo que hace que el tribunal de arbitraje parezca más cercano.
    A primera vista, este acuerdo, que excluye el uso de balas explosivas, parecería favorecer a quienes creen que cada incremento en el peligro de guerra es la mejor esperanza para su cese definitivo. Se dice que un famoso estadista estadounidense incluso atribuyó a la invención de la pólvora el respaldo de su afirmación, argumentando que cada descubrimiento en el arte de la guerra posee, desde esa perspectiva, un efecto salvador y promotor de la paz. Sin embargo, es difícil concebir una ilusión mayor, ya que toda la historia de la guerra evidencia un incremento constante de sus dolores y peligros a medida que han surgido armas de destrucción más efectivas. La ilusión se refuta mejor al analizar los hechos que se exponen a continuación.
    '''
    -----------
    Recordatorio: No menciones el texto, la información proporcionada, los párrafos, la obra ni el autor. Cualquier pregunta sobre el autor debe reformularse para dirigirse a la persona que responde ("tú").
    """
},
{
  "role": "assistant",
  "content": """
    **PREGUNTA:**
    ¿Qué es la Declaración de San Petersburgo y qué problema plantea para quienes apoyan la guerra?

    **RESPUESTA:**
    La Declaración de San Petersburgo es un acuerdo entre naciones que puede expandirse indefinidamente. Plantea un problema para quienes apoyan la guerra, ya que establece un precedente para hacer cumplir los acuerdos internacionales, lo que podría derivar en la creación de un tribunal de arbitraje y, en consecuencia, en la reducción de los conflictos bélicos.

    **PREGUNTA:**
    ¿Cuál es el argumento principal en contra de la idea de que un aumento en el peligro de la guerra conduce a su cese definitivo?

    **RESPUESTA:**
    El argumento principal es que la historia de la guerra ha mostrado un incremento continuo en los sufrimientos y peligros a medida que se desarrollan armas de destrucción cada vez más efectivas. Esto sugiere que, en lugar de detener la guerra, un mayor peligro tiende a prolongarla y a intensificarla.

    **PREGUNTA:**
    ¿Cuál es tu opinión sobre la idea de que cada descubrimiento en el arte de la guerra tiene una influencia salvadora y promotor de la paz?

    **RESPUESTA:**
    Considero que esa idea es una ilusión. La evidencia histórica indica que los avances en armas de destrucción han generado más sufrimiento y peligro, en lugar de promover la paz o detener la guerra.
    """
}, 
{
    "role": "user",
    "content": """Texto del que se generarán preguntas:
    '''
    Size Limit es una herramienta de presupuesto de rendimiento para JavaScript. Revisa cada commit en la integración continua (CI), calcula el costo real de tu código JS para los usuarios finales y lanza un error si dicho costo supera el límite establecido.

    Soporta ES modules y tree-shaking.
    Se puede integrar en GitHub Actions, Circle CI u otro sistema de CI para detectar si una solicitud de extracción añade una dependencia excesiva.
    Es modular, adaptándose tanto a aplicaciones JS grandes que utilizan su propio empaquetador, como a pequeñas librerías npm con numerosos archivos.
    Permite calcular el tiempo que tardaría un navegador en descargar y ejecutar tu JS; una métrica que resulta mucho más precisa y comprensible que el tamaño en bytes, ya que los cálculos incluyen todas las dependencias y polyfills empleados.
    Cuenta con una interfaz de línea de comandos (CLI).

    Con GitHub Actions, Size Limit publicará los cambios en el tamaño del bundle como un comentario en la discusión de la solicitud de extracción.

    Con la opción --why, Size Limit puede explicar por qué tu librería tiene ese tamaño y mostrar el costo real de todas las dependencias internas, utilizando para ello Statoscope en el análisis.
    '''
    -----------
    Recordatorio: No menciones el texto, la información proporcionada, los párrafos, la obra ni el autor. Cualquier pregunta sobre el autor debe reformularse para dirigirse a la persona que responde ("tú").
    
    """
},
{
    "role": "assistant",
    "content": """
    **PREGUNTA:**
    ¿Qué es Size Limit?
    
    **RESPUESTA:**
    Size Limit es una herramienta de presupuesto de rendimiento para JavaScript que revisa cada commit en la CI, calcula el costo real de tu código para los usuarios finales y lanza un error si dicho costo supera el límite establecido.
    
    **PREGUNTA:**
    ¿Puedes enumerar cuatro características de Size Limit?
    
    **RESPUESTA:**
    Size Limit posee diversas características, entre las que destacan:
    
    * Soporta ES modules y tree-shaking.
    * Se integra en sistemas de CI como GitHub Actions o Circle CI para detectar si una solicitud de extracción añade una dependencia excesiva.
    * Es modular, adaptándose tanto a grandes aplicaciones JS que utilizan su propio empaquetador, como a pequeñas librerías npm con numerosos archivos.
    * Calcula el tiempo que tardaría un navegador en descargar y ejecutar tu JS, ofreciendo una métrica más precisa y comprensible que el tamaño en bytes, ya que incluye todas las dependencias y polyfills.
    
    **PREGUNTA:**
    ¿Cuenta Size Limit con una interfaz de línea de comandos (CLI)?
    
    **RESPUESTA:**
    Sí, Size Limit dispone de una CLI.
    
    **PREGUNTA:**
    ¿Qué función tiene la opción `--why` en Size Limit?
    
    **RESPUESTA:**
    La opción `--why` permite que Size Limit explique por qué tu librería alcanza ese tamaño, mostrando el costo real de todas las dependencias internas, utilizando Statoscope para dicho análisis.
    """
}
]
def parse_qa(content):
    """
    Esta función recibe un string con el formato:
    
    **PREGUNTA:**
    <texto de la pregunta>
    
    **RESPUESTA:**
    <texto de la respuesta>
    
    Y retorna una lista de diccionarios con la estructura:
    [
        {"question": "<texto de la pregunta>", "answer": "<texto de la respuesta>"},
        ...
    ]
    """
    pattern = r"\*\*PREGUNTA:\*\*\s*(.*?)\s*\*\*RESPUESTA:\*\*\s*(.*?)(?=\*\*PREGUNTA:\*\*|$)"
    matches = re.findall(pattern, content, re.DOTALL)
    qa_list = [{"question": q.strip(), "answer": a.strip()} for q, a in matches]
    return qa_list
    
def gen_qa_pairs(paragraph):
    message = {
        "role": "user",
        "content": f"""Texto para generar preguntas:  
        '''  
        {paragraph}  
        '''  
        -----------  
        Recordatorio: No menciones el texto, la información proporcionada, los párrafos, la obra ni el autor. Cualquier pregunta sobre el autor debe reformularse para referirse a la persona que responde (tú).  
        """
    }

    groq_client = Groq(api_key=GROQAPIKEY)
    response = groq_client.chat.completions.create(
        messages=[*qa_generation_prompts, message],
        model='llama3-70b-8192',
    )

    contenido = response.choices[0].message.content
    print("\n\gen_qa_pairs.py contenido: \n\n", contenido, "\n\nfin de gen_qa_pairs.py contenido")
    return parse_qa(contenido)


    


In [41]:
print(chunks[50])

Requerimientos no funcionales Técnicas para validación de requerimientos
1. Validación de expertos Personas con experiencia revisan los requerimientos y aprueban o rechazan el requerimiento
2. Prototipado de interfaz de usuario El prototipado de interfaz de usuario es una técnica de representación aproximada de la interfaz de usuario
Los dos tipos principales de prototipos de interfaz de usuario son:
•
Desechables: se utilizan sólo para la validación de los requisitos y posteriormente se desechan.
Pueden ser prototipos en papel o en software.
•
Evolutivos: una vez utilizados para la validación de los requisitos, se mejora su calidad y se
convierten progresivamente en el producto final.
3. Recorrido de BPM: Hacer un BPM que muestre todo el proceso, algoritmo de alto nivel


In [8]:
gen_qa_pairs(chunks[50])


\gen_qa_pairs.py contenido: 

 **PREGUNTA:**
 ¿Cuál es el propósito de la validación de expertos en la técnica de validación de requerimientos?

**RESPUESTA:**
El propósito de la validación de expertos es que personas con experiencia revisen los requerimientos y aprueben o rechazan el requerimiento.

**PREGUNTA:**
 ¿Cuáles son los dos tipos principales de prototipos de interfaz de usuario?

**RESPUESTA:**
Los dos tipos principales de prototipos de interfaz de usuario son: desechables (utilizados solo para la validación de requisitos y posteriormente desechados) y evolutivos (mejorados progresivamente para convertirse en el producto final).

**PREGUNTA:**
 ¿Qué es un recorrido de BPM en la técnica de validación de requerimientos?

**RESPUESTA:**
Un recorrido de BPM es una representación del proceso que muestra todo el proceso, algoritmo de alto nivel. 

fin de gen_qa_pairs.py contenido


[{'question': '¿Cuál es el propósito de la validación de expertos en la técnica de validación de requerimientos?',
  'answer': 'El propósito de la validación de expertos es que personas con experiencia revisen los requerimientos y aprueben o rechazan el requerimiento.'},
 {'question': '¿Cuáles son los dos tipos principales de prototipos de interfaz de usuario?',
  'answer': 'Los dos tipos principales de prototipos de interfaz de usuario son: desechables (utilizados solo para la validación de requisitos y posteriormente desechados) y evolutivos (mejorados progresivamente para convertirse en el producto final).'},
 {'question': '¿Qué es un recorrido de BPM en la técnica de validación de requerimientos?',
  'answer': 'Un recorrido de BPM es una representación del proceso que muestra todo el proceso, algoritmo de alto nivel.'}]

In [42]:
STATE_FILE = r"D:\Jupyter Notebooks\finetuning\generated_qa\generated_qa.json"

def load_state(filepath):
    """
    Carga el estado desde el archivo JSON.
    Si el archivo no existe, retorna un estado inicial.
    """
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8") as file:
            return json.load(file)
    else:
        return {"currentChunkIdx": 0, "qaPairs": []}

def save_state(state, filepath):
    """
    Guarda el estado en el archivo JSON.
    """
    with open(filepath, "w", encoding="utf-8") as file:
        json.dump(state, file, ensure_ascii=False, indent=4)

def process_chunks(chunks, state_filepath=STATE_FILE):
    """
    Procesa una lista de chunks, generando pares de preguntas y respuestas para cada uno.
    El estado (índice del chunk actual y lista de qaPairs) se guarda en un archivo JSON para 
    evitar perder progreso en caso de error (por ejemplo, por límites de rate).
    
    Para cada chunk, se invoca la función gen_qa_pairs, se añaden los nuevos pares al estado
    y se actualiza el índice actual.
    
    En caso de error, se espera 60 segundos antes de reintentar el mismo chunk.
    """
    state = load_state(state_filepath)
    start_idx = state["currentChunkIdx"] + 1
    
    for idx in range(start_idx, len(chunks)):
        chunk = chunks[idx]
        retry_delay = 5

        while True:
            try:
                new_qa_pairs = gen_qa_pairs(chunk)
                state["qaPairs"].extend(new_qa_pairs)
                state["currentChunkIdx"] = idx
                save_state(state, state_filepath)
                print(f"Procesado chunk {idx}. Estado guardado.")
                break

            except Exception as e:
                error_message = str(e)
                print(f"Error al procesar chunk {idx}: {error_message}")

                if "Limit" in error_message and "TPD" in error_message:
                    print("Se alcanzó el límite de tokens por día. Deteniendo el proceso.")
                    return

                print(f"Reintentando en {retry_delay} segundos...")
                time.sleep(retry_delay)


In [43]:
process_chunks(chunks)


\gen_qa_pairs.py contenido: 

 **PREGUNTA:**
¿Cuál es el propósito de los headers en una respuesta HTTP?

**RESPUESTA:**
Los headers proporcionan información adicional sobre la transferencia en una respuesta HTTP.

**PREGUNTA:**
¿Cuál es la función de la versión del protocolo y el código de estado en una respuesta HTTP?

**RESPUESTA:**
La versión del protocolo y el código de estado indican la versión de HTTP que entiende el cliente y el resultado de la solicitud, respectivamente.

**PREGUNTA:**
¿Cuál es la diferencia principal entre una petición GET y una petición POST?

**RESPUESTA:**
(Tú debes investigar y responder sobre la diferencia entre GET y POST)

**PREGUNTA:**
¿Qué es un Uniform Resource Locator (URL)?

**RESPUESTA:**
Un Uniform Resource Locator (URL) es una dirección única que identifica un recurso en la web.

**PREGUNTA:**
¿Cuál es el propósito de los métodos de petición en una solicitud HTTP?

**RESPUESTA:**
Los métodos de petición, como GET y POST, indican la acción que 

In [9]:
def gen_all_qa_pairs(chunks):
    """
    Recibe una lista de strings (chunks) y para cada uno genera los pares de preguntas-respuestas
    usando la función gen_qa_pairs. Retorna una lista unificada de diccionarios, donde cada diccionario
    contiene las claves 'question' y 'answer'.
    """
    all_qa_pairs = []
    for chunk in chunks:
        qa_pairs = gen_qa_pairs(chunk)
        all_qa_pairs.extend(qa_pairs)
    return all_qa_pairs

def save_qa_pairs_to_json(chunks, filename="D:/Jupyter Notebooks/finetuning/pdf-text-merged/qa_pairs.json"):
    """
    Genera pares de preguntas y respuestas para una lista de chunks y los guarda en un archivo JSON.
    
    :param chunks: Lista de strings, cada uno será procesado por gen_qa_pairs.
    :param filename: Nombre del archivo JSON donde se guardarán los resultados.
    """
    qa_pairs = gen_all_qa_pairs(chunks)

    with open(filename, "w", encoding="utf-8") as file:
        json.dump(qa_pairs, file, ensure_ascii=False, indent=4)

    print(f"Resultados guardados en {filename}")


In [10]:
save_qa_pairs_to_json(chunks)


\gen_qa_pairs.py contenido: 

 **PREGUNTA:**
¿Cuál es tu profesión?

**RESPUESTA:**
Soy un profesor.

**PREGUNTA:**
¿Dónde trabajas?

**RESPUESTA:**
Trabajo en la Facultad de Ingeniería, específicamente en la Escuela de Ingeniería de Sistemas y Computación.

**PREGUNTA:**
¿Cuál es el año en el que tienes una conexión especial?

**RESPUESTA:**
El año 2025 tiene algún significado especial para mí, tal vez estoy relacionado con algún evento o programa que ocurre en ese año. 

fin de gen_qa_pairs.py contenido

\gen_qa_pairs.py contenido: 

 **PREGUNTA:**
¿Cuál es el objetivo general de este curso?

**RESPUESTA:**
El objetivo general es proporcionar las bases conceptuales fundamentales de la ingeniería de software y los elementos metodológicos necesarios para llevar a cabo el desarrollo de aplicaciones de software.

**PREGUNTA:**
¿Cuáles son los objetivos específicos de este curso?

**RESPUESTA:**
Los objetivos específicos son:
* Entender los conceptos fundamentales de la ingeniería de sof

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jpbzbjn9e5kr4mf4v3zcptdj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 498969, Requested 3854. Please try again in 8m7.8136s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}